In [1]:
#!/usr/bin/env python3
"""
Scaled-arithmetic time-to-steady test (CW, −ω)
─────────────────────────────────────────────
Edit K_MULT below, run, read the console stats.
Everything else (plateau rule, grid, ODE) is fixed.
"""

# ─── USER-ADJUSTABLE SCALE FACTOR ───────────────────────────────
K_MULT = 3          # <-- put any constant here (float)
# ────────────────────────────────────────────────────────────────

import itertools, numpy as np, pandas as pd
from math import sin, cos, pi
from scipy.integrate import solve_ivp, quad
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# plateau rule (fixed)
WINDOW_SEC, DT = 10, 0.001
WIN           = int(WINDOW_SEC/DT)
TOL_PLATEAU   = 5e-4

# integration grid
T_MAX = 100.0
TIME_EVAL = np.arange(0.0, T_MAX+DT, DT)

# parameter grid (400 cases)
p1_vals = np.linspace(0,2,10)
p2_vals = np.linspace(0,2,10)
a_vals  = [1.0, 2.0, 3.0, 4.0, 5.0]
b_vals  = [1.0, 2.0, 3.0, 4.0, 5.0]
PARAM_GRID = list(itertools.product(p1_vals, p2_vals, a_vals, b_vals))

# ODE and f(δ)
def single_cell_ode(t, y, p1, p2, a, b):
    θ, ψ = y
    s2, s, c = sin(2*(ψ-θ)), sin(ψ-θ), cos(ψ-θ)
    denom = 2*((a*s)**2 + (b*c)**2)**1.5
    num   = (a*b)*(a**2-b**2)*s2
    return [-1.0 - p1*num/denom,  -p2*s2]

def f_delta(δ, p1, p2, a, b):
    s2, s, c = np.sin(2*δ), np.sin(δ), np.cos(δ)
    denom = 2*((a*s)**2 + (b*c)**2)**1.5
    num   = (a*b)*(a**2-b**2)*s2
    return -p2*s2 - (-1.0 - p1*num/denom)

# zeros in one π-period
def one_period_zeros(p1,p2,a,b,n_scan=20000):
    x = np.linspace(-pi,pi,n_scan); y = f_delta(x,p1,p2,a,b)
    roots=[]
    for i in range(n_scan-1):
        if y[i]*y[i+1] < 0:
            lo,hi = x[i],x[i+1]
            for _ in range(30):
                mid=0.5*(lo+hi)
                (hi,lo)=(mid,lo) if f_delta(lo,p1,p2,a,b)*f_delta(mid,p1,p2,a,b)<0 else (hi,mid)
            roots.append(0.5*(lo+hi))
    roots.sort()
    if not roots: return []
    start=roots[0]
    per=[z for z in roots if start<z<start+pi]
    end=start+pi
    if abs(end-per[-1])>1e-8: per.append(end)
    per.insert(0,start)
    return per

# integrate to plateau
def integrate_to_steady(δ0,p1,p2,a,b):
    θ0,ψ0 = pi/3, pi/3 + δ0
    sol = solve_ivp(lambda t,y: single_cell_ode(t,y,p1,p2,a,b),
                    (0,T_MAX), [θ0,ψ0], t_eval=TIME_EVAL,
                    rtol=1e-6, atol=1e-9)
    if not sol.success: return None,None
    δ = sol.y[1]-sol.y[0]
    for i in range(len(δ)-WIN):
        w=δ[i:i+WIN]
        if np.max(np.abs(w-w[0])) < TOL_PLATEAU:
            return w[0], sol.t[i]
    return None,None

# worker
def run_param_set(params):
    p1,p2,a,b=params
    zeros=one_period_zeros(p1,p2,a,b)
    if len(zeros)<2: return []
    rows=[]
    for zL,zR in zip(zeros[:-1],zeros[1:]):
        for q in (0.25,0.50,0.75):
            δ0=zL+q*(zR-zL)
            sgn=np.sign(f_delta(δ0,p1,p2,a,b))
            if sgn==0: continue
            δ_star = zR if sgn>0 else zL
            Δ=abs(δ_star-δ0)
            I,_ = quad(lambda x: abs(f_delta(x,p1,p2,a,b)),
                       min(δ0,δ_star), max(δ0,δ_star), limit=200)
            if I<1e-8: continue
            T_est = Δ**2 / I
            steady, T_num = integrate_to_steady(δ0,p1,p2,a,b)
            if steady is None: continue
            rows.append(dict(T_num=T_num, T_est_ari=T_est))
    return rows

# main
def main():
    with Pool(cpu_count()) as p:
        rows=[]
        for res in tqdm(p.imap_unordered(run_param_set, PARAM_GRID),
                        total=len(PARAM_GRID), desc="Running"):
            rows.extend(res)
    df=pd.DataFrame(rows)
    df["T_est_scaled"]=K_MULT*df["T_est_ari"]
    df.to_excel("time_to_steady_arithmetic.xlsx", index=False)

    diff=df["T_num"]-df["T_est_scaled"]
    n_gt=(diff>1e-6).sum(); n_lt=(diff<-1e-6).sum(); N=len(df)
    med_abs=diff.abs().median(); q1,q3=diff.abs().quantile([0.25,0.75])
    med_rel=(diff.abs()/df["T_num"]).median()
    within2=((df["T_est_scaled"]/df["T_num"]).between(0.5,2)).mean()

    k_best=np.median(df["T_num"]/df["T_est_ari"])

    print(f"\nSaved → time_to_steady_arithmetic.xlsx  ({N} trajectories)")
    print(f"Plateau rule: window 10 s, tolerance ±{TOL_PLATEAU}")
    print(f"\nCurrent K_MULT = {K_MULT:.3f}  |  Best κ from data = {k_best:.3f}")
    print("\nResults for scaled arithmetic estimator")
    print(f"T_num > T_est : {n_gt}/{N}  ({100*n_gt/N:.2f} %)")
    print(f"T_num < T_est : {n_lt}/{N}  ({100*n_lt/N:.2f} %)")
    print(f"Median |Δt|    = {med_abs:.4f} s  (IQR {q1:.4f}–{q3:.4f})")
    print(f"Median rel err = {100*med_rel:.2f} %")
    print(f"Within factor-2= {100*within2:.2f} %")

if __name__ == "__main__":
    main()


Running: 100%|██████████| 2500/2500 [00:51<00:00, 48.58it/s]



Saved → time_to_steady_arithmetic.xlsx  (11130 trajectories)
Plateau rule: window 10 s, tolerance ±0.0005

Current K_MULT = 3.000  |  Best κ from data = 3.301

Results for scaled arithmetic estimator
T_num > T_est : 6975/11130  (62.67 %)
T_num < T_est : 4155/11130  (37.33 %)
Median |Δt|    = 0.4412 s  (IQR 0.2098–0.9023)
Median rel err = 19.77 %
Within factor-2= 96.78 %
